In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

/tmp/ipykernel_893/95880468.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-07 13:09:46.200536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 13:09:46.200672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 13:09:46.817451: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to r

In [2]:
def load_dataset(csv_path):
    df = pd.read_csv(csv_path, sep= ';')
    required_columns = ['Azimuth', 'Altitude', 'Timestamp']

    if not all (col in df.columns for col in required_columns):
        raise ValueError("Required columns are not present in the DataFrame.")
    
    X = df[['Timestamp','Azimuth','Altitude']].values
    y = df[['Azimuth','Altitude']].values

    return X, y, df

In [3]:
csv_path = 'cleaned/cleaned_data_2014-2024.csv'
X, y, df = load_dataset(csv_path)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state= 42)

In [5]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

In [6]:
timesteps = 1
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

In [7]:
model = Sequential()
model.add(LSTM(units=1024, activation='relu', input_shape=(timesteps, X_train_scaled.shape[1]), return_sequences=True))
model.add(LSTM(units=512, activation='relu', return_sequences=True))
model.add(LSTM(units=256, activation='relu', return_sequences=True))
model.add(LSTM(units=128, activation='relu', return_sequences=True))
model.add(LSTM(units=64, activation='relu', return_sequences=True))
model.add(LSTM(units=32, activation='relu', return_sequences=True))
model.add(LSTM(units=16, activation='relu', return_sequences=True))
model.add(LSTM(units=8, activation='relu'))
model.add(Dense(units=2))

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])  

history = model.fit(X_train_reshaped, y_train_scaled, epochs=100, batch_size=4096, validation_data=(X_test_reshaped, y_test_scaled))

2024-03-07 13:10:01.599146: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 13:10:03.732344: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 13:10:03.732601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 13:10:03.737139: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 13:10:03.737284: I external/local_xla/xla/stream_executor

2024-03-07 13:10:07.590283: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/100


2024-03-07 13:10:18.858990: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe44d339cd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-07 13:10:18.859120: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti with Max-Q Design, Compute Capability 7.5
2024-03-07 13:10:18.992442: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-07 13:10:19.294496: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1709791819.507767    1210 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


103/103 [==============================] - 26s 120ms/step - loss: 0.3153 - mae: 0.4944 - val_loss: 0.3090 - val_mae: 0.4881
Epoch 2/100
103/103 [==============================] - 11s 104ms/step - loss: 0.3027 - mae: 0.4816 - val_loss: 0.2962 - val_mae: 0.4749
Epoch 3/100
103/103 [==============================] - 11s 104ms/step - loss: 0.2895 - mae: 0.4678 - val_loss: 0.2824 - val_mae: 0.4603
Epoch 4/100
103/103 [==============================] - 10s 99ms/step - loss: 0.2744 - mae: 0.4517 - val_loss: 0.2640 - val_mae: 0.4405
Epoch 5/100
103/103 [==============================] - 10s 99ms/step - loss: 0.1255 - mae: 0.2739 - val_loss: 0.0461 - val_mae: 0.1725
Epoch 6/100
103/103 [==============================] - 10s 101ms/step - loss: 0.0390 - mae: 0.1620 - val_loss: 0.0368 - val_mae: 0.1583
Epoch 7/100
103/103 [==============================] - 10s 101ms/step - loss: 0.0365 - mae: 0.1572 - val_loss: 0.0361 - val_mae: 0.1557
Epoch 8/100
103/103 [==============================] - 10s 102

In [8]:
predicted_values_scaled = model.predict(X_test_reshaped)
predicted_values = scaler_y.inverse_transform(predicted_values_scaled)
mse = mean_squared_error(y_test_scaled, predicted_values_scaled)
r2 = r2_score(y_test_scaled, predicted_values_scaled)

accuracy = model.evaluate(X_test_reshaped, y_test_scaled)

3287/3287 [==============================] - 24s 7ms/step - loss: 2.2517e-05 - mae: 0.0038


In [9]:
timestamps_test = X_test[:, 0]
df_results = pd.DataFrame({
    'Timestamp': timestamps_test,
    'Actual_Azimuth': y_test[:, 0],
    'Actual_Altitude': y_test[:, 1],
    'Predicted_Azimuth': predicted_values[:, 0],
    'Predicted_Altitude': predicted_values[:, 1]
})

In [10]:
output_csv_path = 'azimuth_altitude_prediction_results.csv'
df_results.to_csv(output_csv_path, index=False)

In [11]:
print(f"Mean Squared Error on Test Data: {mse}")
print(f"R-squared on Test Data: {r2}")
print(f"Accuracy on Test Data: {accuracy}")
print(f"Results saved to: {output_csv_path}")

Mean Squared Error on Test Data: 2.2517023265190713e-05
R-squared on Test Data: 0.9996852128883238
Accuracy on Test Data: [2.2517018805956468e-05, 0.0037750808987766504]
Results saved to: azimuth_altitude_prediction_results.csv


In [12]:
#model.save('model_save')
#model.save_weights('model_weights.h5')